In [117]:
from sklearn.tree import DecisionTreeClassifier as tree
import pandas as pd
import numpy as np
import seaborn as sns
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf

In [118]:
path_to_directory = "/Users/deronmurphy/Dropbox/diabetes_prediction_dataset.csv"

df = pd.read_csv(path_to_directory)

In [119]:
df.sort_values(by=['diabetes'])

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
65262,Female,34.0,0,0,never,27.00,6.2,126,0
65261,Male,49.0,1,0,never,32.31,6.1,80,0
65260,Male,5.0,0,0,No Info,16.49,6.2,126,0
65259,Female,33.0,0,0,No Info,27.32,5.8,200,0
...,...,...,...,...,...,...,...,...,...
12139,Male,40.0,0,0,former,32.08,6.1,260,1
63814,Male,46.0,0,0,never,24.48,7.0,260,1
93029,Male,69.0,1,0,former,26.39,6.1,260,1
37663,Female,58.0,0,0,never,43.74,7.5,155,1


Dataset was unbalanced in the last homework so I balanced it.

In [120]:
df = df.iloc[80000:]


In [121]:
print(df.shape)

(20000, 9)


In [122]:
df.loc[(df['age'] < 13) & (df['smoking_history'] == "No Info"), 'smoking_history'] = 'never'


In [123]:
(df['gender']).replace(to_replace ="Male",value =0,inplace = True)
(df['gender']).replace(to_replace ="Female",value =1,inplace = True)
(df['gender']).replace(to_replace ="Other",value =2,inplace = True)

(df['smoking_history']).replace(to_replace ="never",value =0,inplace = True)
(df['smoking_history']).replace(to_replace ="not current",value =1,inplace = True)
(df['smoking_history']).replace(to_replace ="former",value =2,inplace = True)
(df['smoking_history']).replace(to_replace ="current",value =3,inplace = True)
(df['smoking_history']).replace(to_replace ="ever",value =4,inplace = True)
(df['smoking_history']).replace(to_replace ="No Info",value =5,inplace = True)

In [124]:
from sklearn.model_selection import train_test_split

X_train, X_test =  train_test_split(df,test_size=0.2)

In [125]:
Y = X_train['diabetes']
# Y = Y.T
X = X_train.copy()
X.drop('diabetes', axis=1, inplace=True)

Y_test = X_test['diabetes']
X_test = X_test.copy()
X_test.drop('diabetes', axis=1, inplace=True)

# X = X.T
# X = tf.convert_to_tensor(X)
# Y = tf.convert_to_tensor(Y)

Task 1: Increasing Max Depth increases accuracy slightly. This is because there is a lot of things to consider in the data. It would be best to not overlook many things

In [155]:
classification_1 = tree()

In [156]:
classification = tree(max_depth = 7)

In [157]:
classification = classification.fit(X,Y)
classification_1 = classification_1.fit(X,Y)

In [160]:
y_pred = classification.predict(X_test)
y_pred_1 = classification_1.predict(X_test)
# Import accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


# Compute test set accuracy  
accuracy_1 = accuracy_score(Y_test,y_pred_1)
accuracy = accuracy_score(Y_test, y_pred)
f1_1 = f1_score(Y_test,y_pred_1)
f1 = f1_score(Y_test,y_pred)
print("Test set accuracy of unmodified parameters: {:.2f}".format(accuracy_1))
print("Test set accuracy of Higher Depth: {:.2f}".format(accuracy))
print(f'F1 score of Higher Depth Tree {f1}')
print(f'F1 score of unmodified parameters: {f1_1}')



Test set accuracy of unmodified arameters: 0.95
Test set accuracy of Higher Depth: 0.97
F1 score of Higher Depth Tree 0.8015414258188824
F1 score of unmodified parameters: 0.7078313253012049


Here we can see increasing the depth slightly is good. If we increase too much, we would overfit. 

Task 2: I decided to use Bagging Classifier with 20 estimators and AdaBoost with 20 estimators as well.

In [129]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
model = BaggingClassifier(n_estimators = 20)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')



In [163]:
from numpy import mean
from numpy import std
print('Accuracy of Train Set for Bagging: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
#Accuracy of Train Set


Accuracy of Train Set for Bagging: 0.972 (0.003)


In [164]:
model.fit(X, Y)

BaggingClassifier(n_estimators=20)

In [166]:
y_pred = model.predict(X_test)
f1 = f1_score(Y_test,y_pred)
print(f'F1 Score of BaggingClassifier: {f1}')
# Model Accuracy, how often is the classifier correct?
print("Accuracy of Test Set for Bagging:",accuracy_score(Y_test, y_pred))
#Test Accuracy

F1 Score of BaggingClassifier: 0.7855855855855856
Accuracy of Test Set for Bagging: 0.97025


In [174]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=20, random_state=0, algorithm='SAMME')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(clf, X, Y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy of Train Set for AdaBoost Classifier: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
#Train Accuracy

Accuracy of Train Set for AdaBoost Classifier: 0.972 (0.003)


In [175]:
clf.fit(X,Y)

AdaBoostClassifier(algorithm='SAMME', n_estimators=20, random_state=0)

In [176]:
y_pred = clf.predict(X_test)
f1 = f1_score(Y_test,y_pred)
print(f'F1 Score of AdaBoostClassifier: {f1}')
print("Accuracy:",accuracy_score(Y_test, y_pred))
#Test Accuracy

F1 Score of AdaBoostClassifier: 0.7992277992277992
Accuracy: 0.974


Task 2 Continued: The two results were similar both in terms of accuracy and F1 score. The adaboost has a slightly better F1 Score than the bagging.

Task 3: Strictly Based on Accuracy Score on the Test Set, it seems as if boosting gives the best results with the decision tree classifier slightly behind it. Just slightly behind that is the bagging classifier. The differences between all of these are slight.


Another metric I decided to use was F1 Score. After some research, F1-Score is the best metric to use for binary classification. F1 Score combines both precision and recall which in a task like ours, are both equally important. Before any manipulation to my dataset, the dataset was extremely inbalanced, leaning towards the data having far less diabetic people than nondiabetic. When using F1-Score, we see that it favors the classification tree more than the boosting. This shows how changing the metric can impact your results.